# Autonomous driving - Car detection

Aprenderá sobre la detección de objetos utilizando el modelo YOLO. Muchas de las ideas de este cuaderno se describen en los dos papers de YOLO: [Redmon et al., 2016](https://arxiv.org/abs/1506.02640) y [Redmon and Farhadi, 2016](https://arxiv.org/abs/1612.08242). Descargar previamente la carpeta [yad2k](https://github.com/allanzelener/YAD2K) a su escritorio y la carpeta [model_data](https://drive.google.com/file/d/1W3rqk19V_iI5_0vbwhIx2y1QwRZonLcC/view?usp=sharing) que contiene los pesos, clases y anchor boxes de YoloV2

**Aprenderás a**:
- Usar la detección de objetos en un conjunto de datos de detección de automóviles
- Tratar con cajas delimitadoras(bounding boxes)

## Librerías

In [6]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PIL
from PIL import ImageFont, ImageDraw, Image
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from yad2k.models.keras_yolo import yolo_head
from yad2k.utils.yolo_utils import draw_boxes, get_colors_for_classes, scale_boxes, read_classes, read_anchors, preprocess_image



## 1. Planteamiento del problema

Estás trabajando en un coche autónomo. Como componente crítico de este proyecto, primero le gustaría construir un sistema de detección de automóviles. Para recopilar datos, instaló una cámara en el capó (es decir, en la parte delantera) del automóvil, que toma imágenes de la carretera cada pocos segundos mientras conduce

<center>
<video width="400" height="200" src="image/nb_images/road_video_compressed2.mp4" type="video/mp4" controls>
</video>
</center>

<caption><center> Fotos tomadas de una cámara montada en automóvil mientras conducen alrededor de Silicon Valley. <br> Dataset provided by <a href="https://www.drive.ai/">drive.ai</a>.
</center></caption>

Reunió todas estas imágenes en una carpeta y las etiquetó dibujando cuadros delimitadores alrededor de cada automóvil que encontró. Aquí hay un ejemplo de cómo se ven sus cuadros delimitadores.

<img src="image/nb_images/box_label.png" style="width:500px;height:250;">
<caption><center> <u><b>Figure 1</u></b>: Definició de un box<br> </center></caption>

Si tiene 80 clases que desea que reconozca el detector de objetos, puede representar la etiqueta de clase $c$ ya sea como un número entero del 1 al 80, o como un vector de 80 dimensiones (con 80 números), uno de cuyos componentes es 1 y el resto de los cuales son 0. 

En este ejercicio, aprenderá cómo "You Only Look Once" (YOLO) **realiza la detección de objetos y luego lo aplicará a la detección de automóviles**. Debido a que el modelo YOLO es computacionalmente muy costoso de entrenar, cargaremos pesos previamente entrenados para que los use.

## 2. YOLO

"You Only Look Once" (YOLO) es un algoritmo popular porque logra un alto accuracy y al mismo tiempo puede ejecutarse en tiempo real. Este algoritmo "Solo se ve una vez" la imagen en el sentido de que requiere solo un paso de propagación hacia adelante a través de la red para hacer predicciones. Después de la `supresión no máxima`, luego genera objetos reconocidos junto con los cuadros delimitadores.

### 2.1 - Detalles del modelo

#### Inputs y outputs
- El **input** es un batch de imagenes, y cada imagen tiene la forma (m, 608, 608, 3)
- El **output** es una lista de cuadros delimitadores junto con las clases reconocidas. Cada cuadro delimitador está representado por 6 números $(p_c, b_x, b_y, b_h, b_w, c)$ como se explicó anteriormente. `Si expandes $c$ en un vector de 80 dimensiones, cada cuadro delimitador se representa mediante 85 números.`

#### Anchor Boxes
* Los Anchor boxes se eligen **mediante la exploración de los datos de entrenamiento** para elegir proporciones razonables de height/width que representen las diferentes clases. Para esta tarea, **se eligieron 5 cuadros de anclaje** (para cubrir las 80 clases) y se almacenaron en el archivo './model_data/yolo_anchors.txt'
* La dimensión para los Anchor boxes es la penúltima dimensión en la codificación: $(m, n_H,n_W,anchors,classes)$.
* La arquitectura de YOLO es: `IMAGE (m, 608, 608, 3) -> DEEP CNN -> ENCODING (m, 19, 19, 5, 85)`.   

#### Encoding
Veamos con mayor detalle lo que representa esta codificación.

<img src="image/nb_images/architecture.png" style="width:700px;height:400;">
<caption><center> <u><b> Figure 2 </u></b>: Arquitectura de codificación para yolo<br> </center></caption>

Si el centro/punto medio de un objeto cae en una celda de cuadrícula, esa celda de cuadrícula es responsable de detectar ese objeto.



Dado que estamos usando 5 *cuadros de anclaje*, cada una de las **celdas de 19 x 19** codifica información sobre 5 cuadros. **Los cuadros de anclaje se definen solo por su ancho y alto**.

Para simplificar, aplanaremos las dos últimas dimensiones de la codificación de la forma (19, 19, 5, 85). Entonces la salida del Deep CNN es (19, 19, 425).

<img src="image/nb_images/flatten.png" style="width:700px;height:400;">
<caption><center> <u><b> Figure 3 </u></b>: Aplanando las últimas dos dimensiones<br> </center></caption>

#### Class score

Ahora, para cada box (de cada celda) calcularemos el siguiente producto por elementos y extraeremos una probabilidad de que la Caja contenga una cierta clase.
El score de la clase es $score_{c,i} = p_{c} \times c_{i}$, $p_{c}$ : la probabilidad de que haya un objeto, $c_{i}$ : la probabilidad de que el objeto sea de cierta clase $c_{i}$.

<img src="image/nb_images/probability_extraction.png" style="width:700px;height:400;">
<caption><center> <u><b>Figure 4 </b></u>: Encuentre la clase detectada por cada caja<br> </center></caption>

##### Ejemplo de la figura 4
* En la figura 4, digamos para el cuadro 1 (celda 1), la probabilidad de que exista un objeto es $p_{1}=0.60$. Entonces, hay un 60% de posibilidades de que exista un objeto en el cuadro 1 (celda 1).
* La probabilidad de que el objeto sea de la clase "categoría 3 (un coche)" es $c_{3}=0,73$.
* La puntuación de la casilla 1 y de la categoría "3" es $score_{1,3}=0,60 \times 0,73 = 0,44$.
* Digamos que calculamos el puntaje para las 80 clases en el cuadro 1 y encontramos que el puntaje para la clase de automóvil (clase 3) es el máximo. Así que asignaremos el puntaje 0.44 y la clase "3" a este cuadro "1".

#### Visualizando clases
Aquí hay una forma de visualizar lo que YOLO está prediciendo en una imagen:
- Para cada una de las celdas de la cuadrícula de 19x19, encuentre el máximo de los scores de probabilidad (tomando un máximo entre las 80 clases, un máximo para cada uno de los 5 cuadros de anclaje).
- Colorea esa celda de cuadrícula de acuerdo con el objeto que esa celda de cuadrícula considere más probable.

Hacer esto da como resultado esta imagen:

<img src="image/nb_images/proba_map.png" style="width:300px;height:300;">
<caption><center> <u><b>Figure 5 </b></u>: Cada una de las celdas de la cuadrícula de 19x19 está coloreada de acuerdo con la clase que tiene la mayor probabilidad prevista en esa celda. <br> </center></caption>

Tenga en cuenta que esta visualización no es una parte central del algoritmo YOLO en sí mismo para hacer predicciones; *es solo una buena forma de visualizar un resultado intermedio del algoritmo.*

#### Visualizing bounding boxes
Otra forma de visualizar la salida de YOLO es trazar los cuadros delimitadores que genera. Hacer eso da como resultado una visualización como esta:

<img src="image/nb_images/anchor_map.png" style="width:200px;height:200;">
<caption><center> <u> Figure 6 </u>: Cada celda te da 5 cajas. En total, el modelo predice: 19x19x5 = 1805 cajas con solo mirar una vez la imagen (¡un paso hacia adelante a través de la red)! Diferentes colores denotan diferentes clases. <br> </center></caption>

#### Non-Max suppression
En la figura anterior, trazamos solo casillas para las que el modelo había asignado una probabilidad alta, pero todavía son demasiadas casillas. Le gustaría reducir la salida del algoritmo a un número mucho menor de objetos detectados.

Para ello, utilizará **non-max suppression**. En concreto, llevarás a cabo estos pasos:
- **Deshágase de las casillas con un score bajo** (lo que significa que la casilla no tiene mucha confianza para detectar una clase, ya sea debido a la baja probabilidad de cualquier objeto o a la baja probabilidad de esta clase en particular).
- Seleccionar solo una casilla cuando varias casillas se superponen entre sí y detectan el mismo objeto.


### 2.2 - Filtrando con un umbral en las class scores
Primero va a aplicar un filtro por umbral. Le gustaría deshacerse de cualquier casilla para la cual el "score" de la clase sea inferior al umbral elegido.

El modelo te da un total de 19x19x5x85 números, con cada casilla descrita por 85 números. Es conveniente reordenar el tensor dimensional (19,19,5,85) (o (19,19,425)) en las siguientes variables:
- `box_confidence`: tensor de forma $(19, 19, 5, 1)$ que contiene $p_c$ (probabilidad de confianza de que haya algún objeto) para cada una de las 5 casillas previstas en cada una de las celdas de 19x19.
- `boxes`: tensor de forma $(19, 19, 5, 4)$ que contiene el punto medio y las dimensiones $(b_x, b_y, b_h, b_w)$ para cada una de las 5 casillas de cada celda.
- `box_class_probs`: tensor de forma $(19, 19, 5, 80)$ que contiene las "probabilidades de clase" $(c_1, c_2, ... c_{80})$ para cada una de las 80 clases para cada una de las 5 casillas por celda.




In [7]:
def yolo_filter_boxes(boxes, box_confidence, box_class_probs, threshold = 0.6):
    """Filtra los cuadros de YOLO al establecer un umbral de confianza de clase y objeto.    
    Arguments:
        boxes -- tensor of shape (19, 19, 5, 4)
        box_confidence -- tensor of shape (19, 19, 5, 1)
        box_class_probs -- tensor of shape (19, 19, 5, 80)
        threshold -- valor real, if [puntuación de probabilidad de clase más alta < umbral], deshágase de la casilla correspondiente """

    box_scores = box_class_probs*box_confidence  # (19,19,5,80)

    # Encuentre box_classes usando max box_scores, realice un seguimiento de la score correspondiente
    box_classes = tf.argmax(box_scores, axis = -1) # (19,19,5)
    box_class_scores = tf.reduce_max(box_scores, axis = -1) # (19,19,5)

    # Cree una máscara de filtrado basada en "box_class_scores" usando "umbral"
    # casillas que desea conservar (con probabilidad >= umbral)
    filtering_mask = (box_class_scores >= threshold) 

    # Aplicar la máscara a box_class_scores, boxes y box_classes
    scores = tf.boolean_mask(box_class_scores, filtering_mask)
    boxes = tf.boolean_mask(boxes, filtering_mask)
    classes = tf.boolean_mask(box_classes, filtering_mask)

    return scores, boxes, classes



In [8]:
tf.random.set_seed(10)
box_confidence = tf.random.normal([19, 19, 5, 1], mean=1, stddev=4, seed = 1)
boxes = tf.random.normal([19, 19, 5, 4], mean=1, stddev=4, seed = 1)
box_class_probs = tf.random.normal([19, 19, 5, 80], mean=1, stddev=4, seed = 1)
scores, boxes, classes = yolo_filter_boxes(boxes, box_confidence, box_class_probs, threshold = 0.5)
print("scores[2] = " + str(scores[2].numpy()))
print("boxes[2] = " + str(boxes[2].numpy()))
print("classes[2] = " + str(classes[2].numpy()))
print("scores.shape = " + str(scores.shape))
print("boxes.shape = " + str(boxes.shape))
print("classes.shape = " + str(classes.shape))

scores[2] = 9.270486
boxes[2] = [ 4.6399336  3.2303846  4.431282  -2.202031 ]
classes[2] = 8
scores.shape = (1789,)
boxes.shape = (1789, 4)
classes.shape = (1789,)


### 2.3 - Non-max Suppression

Incluso después de filtrar por **umbral** sobre las class scores, aún termina con muchas casillas superpuestas. Un segundo filtro para seleccionar las casillas correctas se denomina supresión no máxima (NMS).

<img src="image/nb_images/non-max-suppression.png" style="width:500px;height:400;">
<caption><center> <u> <b>Figure 7</b> </u>: En este ejemplo, el modelo predijo 3 autos, pero en realidad son 3 predicciones del mismo auto. Ejecutar la supresión no máxima (NMS) seleccionará solo la más precisa (probabilidad más alta) de las 3 casillas. <br> </center></caption>

La supresión no máxima utiliza la función muy importante llamada **"Intersection over Union"**, o IoU.

<img src="image/nb_images/iou.png" style="width:500px;height:400;">

<caption><center> <u> Figure 8 </u>: Definición de "Intersection over Union". <br> </center></caption>

***Implementando iou():***
- En este código, usamos la convención de que (0,0) es la esquina superior izquierda de una imagen, (1,0) es la esquina superior derecha y (1,1) es la esquina inferior derecha. En otras palabras, el origen (0,0) comienza en la esquina superior izquierda de la imagen. A medida que x aumenta, nos movemos hacia la derecha. A medida que y aumenta, nos movemos hacia abajo.
- Para este ejercicio, definimos un cuadro usando sus dos esquinas: superior izquierda $(x_1, y_1)$ e inferior derecha $(x_2,y_2)$, en lugar de usar el punto medio, alto y ancho. (Esto hace que sea un poco más fácil calcular la intersección).
- Para calcular el área de un rectángulo, multiplica su altura $(y_2 - y_1)$ por su ancho $(x_2 - x_1)$. (Dado que $(x_1,y_1)$ es la parte superior izquierda y $x_2,y_2$ son la parte inferior derecha, estas diferencias no deberían ser negativas.
- Para encontrar la **intersección** de las dos cajas $(xi_{1}, yi_{1}, xi_{2}, yi_{2})$:
    - Siéntase libre de dibujar algunos ejemplos en papel para aclarar esto conceptualmente.
    - La esquina superior izquierda de la intersección $(xi_{1}, yi_{1})$ se encuentra comparando las esquinas superiores izquierdas $(x_1, y_1)$ de los dos cuadros y encontrando un vértice que tenga una coordenada x que está más cerca de la derecha, y la coordenada y que está más cerca de la parte inferior.
    - La esquina inferior derecha de la intersección $(xi_{2}, yi_{2})$ se encuentra comparando las esquinas inferiores derechas $(x_2,y_2)$ de los dos cuadros y encontrando un vértice cuya coordenada x esté más cerca de la izquierda y la coordenada y que está más cerca de la parte superior.
    - Los dos boxes **pueden no tener intersección**. Puede detectar esto si las coordenadas de intersección que calcula terminan siendo las esquinas superior derecha y/o inferior izquierda de un cuadro de intersección. Otra forma de pensar en esto es si calcula la altura $(y_2 - y_1)$ o el ancho $(x_2 - x_1)$ y encuentra que al menos una de estas longitudes es negativa, entonces no hay intersección (el área de intersección es cero ). 
    - Los dos cuadros pueden intersecarse en los **bordes o vértices**, en cuyo caso el área de intersección sigue siendo cero. Esto sucede cuando la altura o el ancho (o ambos) de la intersección calculada es cero.

In [9]:
def iou(box1, box2):
    """Implement the intersection over union (IoU) between box1 and box2
    Arguments:
    box1 -- first box, list object with coordinates (box1_x1, box1_y1, box1_x2, box1_y2)
    box2 -- second box, list object with coordinates (box2_x1, box2_y1, box2_x2, box2_y2)
    """
    (box1_x1, box1_y1, box1_x2, box1_y2) = box1
    (box2_x1, box2_y1, box2_x2, box2_y2) = box2

    xi1 = max(box1_x1, box2_x1)
    yi1 = max(box1_y1, box2_y1)
    xi2 = min(box1_x2, box2_x2)
    yi2 = min(box1_y2, box2_y2)

    inter_width = max(0, yi2 - yi1)
    inter_height = max(0, xi2 - xi1)
    inter_area = inter_width* inter_height

    box1_area = (box1_x2 - box1_x1)*(box1_y2-box1_y1)
    box2_area = (box2_x2 - box2_x1)*(box2_y2-box2_y1)
    union_area = box1_area + box2_area - inter_area

    # Calculo de IoU
    iou = inter_area/union_area

    return iou

In [10]:
## Caso 1: Interseccion de caajas
box1 = (2, 1, 4, 3)
box2 = (1, 2, 3, 4)

print("iou por cajas que se cruzan = " + str(iou(box1, box2)))
assert iou(box1, box2) < 1, "El área de intersección debe ser siempre menor o igual que el área de unión."
assert np.isclose(iou(box1, box2), 0.14285714), "Valor incorrecto. Verifique su implementación. Problema con las cajas que se cruzan"

## Caso 2: Las cajas no se cruzan
box1 = (1,2,3,4)
box2 = (5,6,7,8)
print("iou para cajas que no se cruzan = " + str(iou(box1,box2)))
assert iou(box1, box2) == 0, "La intersección debe ser 0"

## Caso 3: Las cajas se cruzan solo en los vértices
box1 = (1,1,2,2)
box2 = (2,2,3,3)
print("iou para cajas que solo se tocan en los vértices = " + str(iou(box1,box2)))
assert iou(box1, box2) == 0, "La intersección en los vértices debe ser 0"

## Caso 4: Las cajas se cruzan solo en el borde
box1 = (1,1,3,3)
box2 = (2,3,3,4)
print("iou para cajas que solo se tocan en los bordes = " + str(iou(box1,box2)))
assert iou(box1, box2) == 0, "La intersección en los bordes debe ser 0"


iou por cajas que se cruzan = 0.14285714285714285
iou para cajas que no se cruzan = 0.0
iou para cajas que solo se tocan en los vértices = 0.0
iou para cajas que solo se tocan en los bordes = 0.0


### 2.4 - YOLO Non-max Suppression

Ahora está listo para implementar la non-max suppression. Los pasos clave son:
1. Seleccione la casilla que tenga la score más alta.
2. Calcule la superposición de este cuadro con todos los demás cuadros y elimine los cuadros que se superponen significativamente (iou >= `iou_threshold`).
3. Vuelva al paso 1 e itere hasta que no haya más casillas con una score más baja que la casilla seleccionada actualmente.
Esto eliminará todos los cuadros que tengan una gran superposición con los cuadros seleccionados. Solo quedan las *"mejores"* cajas.

***Implemente yolo_non_max_suppression()*** usando TensorFlow. 

TensorFlow tiene dos funciones integradas que se usan para implementar la supresión no máxima (por lo que en realidad no necesita usar su implementación `iou()`):

[tf.image.non_max_suppression()](https://www.tensorflow.org/api_docs/python/tf/image/non_max_suppression) Elimina las cajas que tienen una gran superposición de intersección sobre unión (IOU) con las cajas previamente seleccionadas. Los cuadros delimitadores se proporcionan como [y1, x1, y2, x2], donde (y1, x1) y (y2, x2) son las coordenadas de cualquier par diagonal de esquinas del cuadro y las coordenadas se pueden proporcionar normalizadas (es decir, dentro del intervalo [0, 1]) o absoluta. 

Devuelve un tensor entero de forma [M] que representa los índices seleccionados del tensor de cajas, donde M <= max_output_size

``` python
tf.image.non_max_suppression(
    boxes,
    scores,
    max_output_size,
    iou_threshold=0.5,
    name=None
)
```

[tf.gather()](https://www.tensorflow.org/api_docs/python/tf/gather)
Reúne slices de `reference` de acuerdo con los `indices`, indices debe ser un tensor entero de cualquier dimensión (a menudo 1-D).

``` python
tf.gather(
    reference,
    indices
)
```

In [11]:
def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
    """
    Applies Non-max suppression (NMS) to set of boxes
    
    Arguments:
    scores -- tensor of shape (None,), output of yolo_filter_boxes()
    boxes -- tensor of shape (None, 4), output of yolo_filter_boxes() that have been scaled to the image size (see later)
    classes -- tensor of shape (None,), output of yolo_filter_boxes()
    max_boxes -- integer, maximum number of predicted boxes you'd like
    iou_threshold -- real value, "intersection over union" threshold used for NMS filtering
    """

    max_boxes_tensor = tf.cast(max_boxes, dtype = 'int32')

    # Para obtener la lista de índices correspondientes a las mejores cajas que guardaremos
    nms_indices = tf.image.non_max_suppression(boxes, scores,
                    max_output_size = max_boxes_tensor, iou_threshold = iou_threshold)

    # Use tf.gather() para seleccionar los scores, boxes y classes de los nms_indices
    scores = tf.gather(scores, nms_indices)
    boxes = tf.gather(boxes, nms_indices)
    classes = tf.gather(classes, nms_indices)

    return scores, boxes, classes
    
    

In [12]:
tf.random.set_seed(10)
scores = tf.random.normal([54,], mean=1, stddev=4, seed = 1)
boxes = tf.random.normal([54, 4], mean=1, stddev=4, seed = 1)
classes = tf.random.normal([54,], mean=1, stddev=4, seed = 1)
scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes)


In [13]:
print("boxes = " + str(len(boxes)))
print("scores[2] = " + str(scores[2].numpy()))
print("boxes[2] = " + str(boxes[2].numpy()))
print("classes[2] = " + str(classes[2].numpy()))
print("scores.shape = " + str(scores.numpy().shape))
print("boxes.shape = " + str(boxes.numpy().shape))
print("classes.shape = " + str(classes.numpy().shape))

boxes = 10
scores[2] = 8.147684
boxes[2] = [ 6.0797963   3.743308    1.3914018  -0.34089637]
classes[2] = 1.7079165
scores.shape = (10,)
boxes.shape = (10, 4)
classes.shape = (10,)


### 2.5 - Terminando el filtrado

Es hora de implementar una **función que tome la salida de la CNN profunda** (la codificación dimensional 19x19x5x85) y **filtre todas las casillas** usando las funciones que acaba de implementar.

***Implemente `yolo_eval()`*** que toma la salida de la codificación de YOLO y filtra las boxes usando el umbral de score y NMS. Solo hay un último detalle de implementación que debe conocer. Hay algunas formas de representar cajas, como a través de sus esquinas o a través de su punto medio y alto/ancho. YOLO convierte entre algunos de estos formatos en diferentes momentos, utilizando las siguientes funciones (que hemos proporcionado):

```python
boxes = yolo_boxes_to_corners(box_xy, box_wh) 
```
convierte las coordenadas de la caja de yolo (x,y,w,h) en las coordenadas de las esquinas de la caja (y1, x1, y2, x2) para ajustarse a la entrada de `yolo_filter_boxes`

```python
boxes = scale_boxes(boxes, image_shape)
```
La red de YOLO fue entrenada para funcionar con imágenes de 608x608. Si está probando estos datos en una imagen de diferente tamaño, por ejemplo, el conjunto de datos de detección de automóviles tenía imágenes de 720x1280, `este paso vuelve a escalar los cuadros para que se puedan trazar sobre la imagen original de 720x1280.`

No te preocupes por estas dos funciones; le mostraremos dónde deben ser llamados.

In [14]:
def yolo_boxes_to_corners(box_xy, box_wh):
    """Convertir predicciones de caja de YOLO en esquinas de cuadro delimitador."""    
    # box_xy (19x19x5x2)
    # box_wh (19x19x5x2)

    box_mins = box_xy - (box_wh / 2.)
    box_maxes = box_xy + (box_wh / 2.)

    return tf.concat([box_mins[...,1:2],
                      box_mins[...,0:1],
                      box_maxes[..., 1:2],
                      box_maxes[..., 0:1]], axis = -1)

In [15]:
def scale_boxes(boxes, image_shape):
    """ Escala los boxes predichos para poder dibujarlos en la imagen"""
    height = image_shape[0]
    width = image_shape[1]
    image_dims = tf.cast([height, width]*2, dtype = 'float32')
    image_dims = tf.reshape(image_dims, [1,4])
    boxes = boxes * image_dims # Forma de boxes (19x19x5x4)
    return boxes

In [16]:
def yolo_eval(yolo_outputs, image_shape = (720, 1280), max_boxes=10, score_threshold=.6, iou_threshold=.5):
    """Convierte el output de codificacion de YOLO (muchos cuadros) en sus boxes predichos junto con sus scores, coordenadas de caja y clases.
    Arguments:
    Para una imagen
    yolo_outputs -- output of the encoding model (for image_shape of (608, 608, 3)), contains 4 tensors:
                    box_xy: tensor of shape (None, 19, 19, 5, 2)
                    box_wh: tensor of shape (None, 19, 19, 5, 2)
                    box_confidence: tensor of shape (None, 19, 19, 5, 1)
                    box_class_probs: tensor of shape (None, 19, 19, 5, 80)
    image_shape -- tensor de forma (2,) que contiene la forma del input, en este notebook usamos (608., 608.) (tiene que ser dtype float32)
    max_boxes -- integer, maximum number of predicted boxes you'd like
    score_threshold -- real value, if [ highest class probability score < threshold], then get rid of the corresponding box
    iou_threshold -- real value, "intersection over union" threshold used for NMS filtering
    
    Returns:
    scores -- tensor of shape (None, ), predicted score for each box
    boxes -- tensor of shape (None, 4), predicted box coordinates
    classes -- tensor of shape (None,), predicted class for each box
    """

    #Recuperar salidas del modelo YOLO
    box_xy, box_wh, box_confidence, box_class_probs = yolo_outputs

    # Convierta los boxes para que estén listos para las funciones de filtrado (convierta los cuadros box_xy y box_wh en coordenadas de esquina)
    boxes = yolo_boxes_to_corners(box_xy, box_wh)

    # Use una de las funciones que ha implementado para realizar el filtrado de score con un umbral de score_threshold
    scores, boxes, classes = yolo_filter_boxes(boxes, box_confidence, box_class_probs, score_threshold)

    # Escale los boxes a la forma original de la imagen (720x1280 o lo que sea)
    # La red fue entrenada para ejecutarse en imágenes de 608x608 
    boxes = scale_boxes(boxes, image_shape)

    # Use una de las funciones que ha implementado para realizar la supresión no máxima con
    # número máximo de boxes establecido en max_boxes y un umbral de iou_threshold
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes, iou_threshold)

    return scores, boxes, classes



In [17]:
tf.random.set_seed(10)
yolo_outputs = (tf.random.normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                tf.random.normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                tf.random.normal([19, 19, 5, 1], mean=1, stddev=4, seed = 1),
                tf.random.normal([19, 19, 5, 80], mean=1, stddev=4, seed = 1))

In [19]:
scores, boxes, classes = yolo_eval(yolo_outputs)
print("scores[2] = " + str(scores[2].numpy()))
print("boxes[2] = " + str(boxes[2].numpy()))
print("classes[2] = " + str(classes[2].numpy()))
print("scores.shape = " + str(scores.numpy().shape))
print("boxes.shape = " + str(boxes.numpy().shape))
print("classes.shape = " + str(classes.numpy().shape))


scores[2] = 171.60194
boxes[2] = [-1240.3483 -3212.5881  -645.78    2024.3052]
classes[2] = 16
scores.shape = (10,)
boxes.shape = (10, 4)
classes.shape = (10,)


## 3 - Pruebe el modelo preentrenado de YOLO en imágenes

En esta parte, utilizará un modelo previamente entrenado y lo probará en el conjunto de datos de detección de automóviles.

### 3.1 - Definición de clases, anchors y forma de imagen

Está tratando de detectar 80 clases y está utilizando 5 cuadros de anclaje(anchor boxes). La información de las 80 clases y 5 cajas está reunida en dos archivos: "coco_classes.txt" y "yolo_anchors.txt". Leerá los nombres de las clases y los anchors de los archivos de texto. `El conjunto de datos de detección de automóviles tiene imágenes de 720x1280`, que **se procesan previamente en imágenes de 608x608**.

In [20]:
class_names = read_classes("model_data/coco_classes.txt")
anchors = read_anchors("model_data/yolo_anchors.txt")
model_image_size = (608, 608) # Igual que el tamaño de capa de entrada de yolo_model

In [22]:
anchors

array([[0.57273 , 0.677385],
       [1.87446 , 2.06253 ],
       [3.33843 , 5.47434 ],
       [7.88282 , 3.52778 ],
       [9.77052 , 9.16828 ]])

### 3.2 - Cargando un modelo preentrenado

* Entrenar un modelo YOLO lleva mucho tiempo y requiere un conjunto de datos bastante grande de cuadros delimitadores etiquetados para una amplia gama de clases objetivo.
* Va a cargar un modelo Keras YOLO preentrenado existente almacenado en "yolo.h5".
* Estos pesos provienen del sitio web oficial de YOLO y se convirtieron mediante una función escrita por Allan Zelener. Las referencias se encuentran al final de este cuaderno. Técnicamente, estos son los parámetros del modelo "YOLOv2", pero simplemente nos referiremos a él como "YOLO" en este cuaderno.

Ejecute la celda a continuación para cargar el modelo desde este archivo.

In [ ]:
yolo_model = load_model('model_data/yolo.h5', compile = False)

Esto carga los pesos de un modelo YOLO entrenado. Aquí hay un resumen de las capas que contiene su modelo:

In [ ]:
yolo_model.summary()

**Recordar:** este modelo convierte un lote preprocesado de imágenes de entrada (forma: (m, 608, 608, 3)) en un tensor de forma (m, 19, 19, 5, 85) como se explica en la Figura (2).

### 3.3 - Convierta la salida del modelo en tensores de cuadro delimitador utilizables

La salida de `yolo_model` es un tensor (m, 19, 19, 5, 85) que necesita pasar por un procesamiento y una conversión. La siguiente celda hace eso por ti.
Si tiene curiosidad acerca de cómo se implementa `yolo_head`, puede encontrar la definición de la función en el archivo ['keras_yolo.py'](https://github.com/allanzelener/YAD2K/blob/master/yad2k/models/keras_yolo.py).  El archivo se encuentra en su espacio de trabajo en esta ruta 'yad2k/models/keras_yolo.py'.

In [ ]:
image, image_data = preprocess_image('./images/test.jpg', model_image_size = (608, 608))
yolo_model_outputs = yolo_model(image_data)

In [ ]:
yolo_outputs = yolo_head(yolo_model_outputs, anchors, len(class_names))

La variable `yolo_outputs` se definirá como un conjunto de 4 tensores que luego puede usar como entrada para su función `yolo_eval`.

### 3.4 - Filtrando cajas

`yolo_outputs` te dio todos los cuadros predichos de `yolo_model` en el formato correcto. Ahora está listo para realizar el filtrado y seleccionar solo las mejores casillas. Llamemos ahora a `yolo_eval`, que habías implementado previamente, para hacer esto.

In [ ]:
scores, boxes, classes = yolo_eval(yolo_outputs)

### 3.5 - Ejecutar el gráfico en una imagen

Ha creado un gráfico que se puede resumir de la siguiente manera:

1. <font color='purple'> yolo_model.input </font> se le da a `yolo_model`. El modelo se utiliza para calcular la salida. <font color='purple'> yolo_model.output </font>
2. <font color='purple'> yolo_model.output </font> es procesado por `yolo_head`. Te dá <font color='purple'> yolo_outputs </font>
3. <font color='purple'> yolo_outputs </font> pasa por una función de filtrado, `yolo_eval`. Muestra sus predicciones: <font color='purple'> scores, boxes, classes </font>

Ahora, hemos implementado para usted la función `predict(image_file)`, que ejecuta el gráfico para probar YOLO en una imagen para calcular `out_scores`, `out_boxes`, `out_classes`.
El siguiente código también usa la siguiente función:
```python
image, image_data = preprocess_image("images/" + image_file, model_image_size = (608, 608))
```
que abre el archivo de imagen y escala, remodela y normaliza la imagen. Devuelve las salidas:

- image: una representación de python (PIL) de su imagen utilizada para dibujar cuadros. No necesitarás usarlo (imagen previa)
- image_data: Un numpy-array representando la imagen. Esta será la entrada a la CNN. (array, de la imagen reformada)

In [24]:
def predict(image_file):
    """
     Ejecuta el gráfico para predecir cuadros para "image_file". Imprime y grafica las predicciones.
    
     Argumentos:
     image_file: nombre de una imagen almacenada en la carpeta "image/images".
    
     Devoluciones:
     out_scores -- tensor de forma (None, ), puntajes de los cuadros predichos
     out_boxes -- tensor de forma (None, 4), coordenadas de los cuadros predichos
     out_classes -- tensor de forma (None, ), índice de clase de los cuadros predichos
    
     Nota: "None" en realidad representa el número de cajas pronosticadas, varía entre 0 y max_boxes.
     """

    # Preprocesando la imagen
    image, image_data = preprocess_image("image/images/" + image_file, model_image_size = (608,608))

    # Su salida es de forma (1, 19, 19, 5, 85)
    yolo_model_outputs = yolo_model(image_data)

    # Pero yolo_eval toma como entrada un tensor que contiene 4 tensores: box_xy, box_wh, box_confidence & box_class_probs
    yolo_outputs = yolo_head(yolo_model_outputs, anchors, len(class_names))

    out_scores, out_boxes, out_classes = yolo_eval(yolo_outputs, [image.size[1],  image.size[0]], 10, 0.3, 0.5)

    print('Se encontró {} cajas para {}'.format(len(out_boxes), "image/images/" + image_file))
    # Genere colores para dibujar cuadros delimitadores.
    colors = get_colors_for_classes(len(class_names))
    
    # Dibujar cuadros delimitadores en el archivo de imagen
    # draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors, image_shape)
    draw_boxes(image, out_scores, out_boxes, out_classes ,class_names, colors)

    # Guardar el cuadro delimitador predicho en la imagen
    image.save(os.path.join("out", str(image_file).split('.')[0] + "_annotated." + str(image_file).split('.')[1]), quality = 100)

    # Mostrar los resultados en el notebook
    output_image = Image.open(os.path.join("out", str(image_file).split('.')[0] + "_annotated." + str(image_file).split('.')[1]))
    plt.imshow(output_image)

    return out_scores, out_boxes, out_classes

Ejecute la siguiente celda en la imagen "test.jpg" para verificar que su función sea correcta.

In [ ]:
out_scores, out_boxes, out_classes = predict("0001.jpg")

Si tuviera que ejecutarlo en un bucle for sobre todas sus imágenes. Esto es lo que obtendrías:

<center>
<video width="400" height="200" src="image/nb_images/pred_video_compressed2.mp4" type="video/mp4" controls>
</video>
</center>

<caption><center> Predicciones del modelo YOLO en imágenes tomadas con una cámara mientras conduce por Silicon Valley <br> [drive.ai](https://www.drive.ai/) </center></caption>


## <font color='green'>Lo que debes recordar:
    
- YOLO es un modelo de detección de objetos de última generación que es rápido y preciso
- Ejecuta una imagen de entrada a través de una CNN que genera un volumen dimensional de 19x19x5x85.
- La codificación(encoding) se puede ver como una cuadrícula donde cada una de las celdas de 19x19 contiene información sobre 5 casillas.
- Filtra a través de todas las casillas utilizando supresión no máxima. Específicamente: 
    - Score thresholding en la probabilidad de detectar una clase para mantener solo casillas precisas (alta probabilidad)
    - Umbral de intersección sobre unión (IoU) para eliminar cajas superpuestas
- Debido a que entrenar un modelo YOLO a partir de pesos inicializados aleatoriamente no es trivial y requiere un gran conjunto de datos, así como muchos cálculos, en este ejercicio usamos parámetros de modelo previamente entrenados. Si lo desea, también puede intentar ajustar el modelo YOLO con su propio conjunto de datos, aunque este no sería un ejercicio trivial.

**Referencias**: Las ideas presentadas en este cuaderno provienen principalmente de los dos artículos de YOLO. La implementación aquí también tomó una inspiración significativa y usó muchos componentes del repositorio GitHub de Allan Zelener. Los pesos previamente entrenados utilizados en este ejercicio provienen del sitio web oficial de YOLO.
- Joseph Redmon, Santosh Divvala, Ross Girshick, Ali Farhadi - [You Only Look Once: Unified, Real-Time Object Detection](https://arxiv.org/abs/1506.02640) (2015)
- Joseph Redmon, Ali Farhadi - [YOLO9000: Better, Faster, Stronger](https://arxiv.org/abs/1612.08242) (2016)
- Allan Zelener - [YAD2K: Yet Another Darknet 2 Keras](https://github.com/allanzelener/YAD2K)
- The official YOLO website (https://pjreddie.com/darknet/yolo/) 